In [1]:
pip install pytorch-tabnet torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool

# TabNet
try:
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier
    TABNET_AVAILABLE = True
except:
    print("⚠️ TabNet not available. Install: pip install pytorch-tabnet torch")
    TABNET_AVAILABLE = False


# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"
OUT_PATH   = "../outputs/06_ML&DL_ensemble_all.csv"

if not os.path.exists(TRAIN_PATH) and os.path.exists("/mnt/data/train.csv"):
    TRAIN_PATH = "/mnt/data/train.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"
SUB_ID_COL = sub.columns[0]
SUB_PRED_COL = sub.columns[1]


# ============================================================
# 1) Feature engineering
# ============================================================
EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

# 🆕 고결측 변수 제거 리스트 (99%+)
HIGH_MISSING_COLS = [
    '난자 해동 경과일',           # 99.4%
    'PGS 시술 여부',             # 99.2%
    'PGD 시술 여부',             # 99.1%
    '착상 전 유전 검사 사용 여부', # 98.9%
    '임신 시도 또는 마지막 임신 경과 연수',  # 96.3%
]

MISS_FLAG_COLS = ["배아 이식 경과일", "난자 채취 경과일", "배아 해동 경과일"]

def safe_div(a, b):
    return np.where(b == 0, 0.0, a / b)

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # 🆕 고결측 변수 제거
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    print(f"✅ 고결측 변수 {len(HIGH_MISSING_COLS)}개 제거")

    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        x = str(x)
        if "IUI" in x:  return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x:  return "IVF"
        if "DI" in x:   return "DI"
        return "Other"

    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)

    # 보조기술 토큰
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    df["AH_포함"]         = s.str.contains("AH", na=False).astype(int)

    # 배아 단계 결측 패턴
    df["배아_stage_missing_count"] = df[EMBRYO_STAGE_COLS].isna().sum(axis=1)
    df["배아_stage_all_missing"]   = (df["배아_stage_missing_count"] == len(EMBRYO_STAGE_COLS)).astype(int)
    df["배아_이식_여부"]           = 1 - df["배아_stage_all_missing"]

    # 총시술_bin3
    def collapse_trials(x):
        if pd.isna(x):
            return "Unknown"
        if x == "0회":
            return "0회"
        if x in ["1회", "2회"]:
            return "1-2회"
        return "3회 이상"
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)

    # 나이_3구간
    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음":
            return "Unknown"
        if age == "만18-34세":
            return "34세 이하"
        if age in ["만35-37세", "만38-39세"]:
            return "35-39세"
        return "40세 이상"
    df["나이_3구간"] = df["시술 당시 나이"].apply(age_group_simple)

    # Day5 이식 여부
    d = pd.to_numeric(df["배아 이식 경과일"], errors="coerce")
    df["Day5_이식_여부"] = (d == 5).astype(int)

    # 불임 원인 개수
    tmp = df[INFERTILITY_COLS].apply(pd.to_numeric, errors="coerce").fillna(0)
    df["불임_원인_개수"] = tmp.sum(axis=1)

    # 전체 결측 개수
    df["전체_missing_count"] = df.isna().sum(axis=1)

    # 결측 flag
    for c in MISS_FLAG_COLS:
        if c in df.columns:
            df[f"{c}_isna"] = df[c].isna().astype(int)

    # 숫자형 안전 변환
    num_candidates = [
        "총 생성 배아 수", "이식된 배아 수", "저장된 배아 수", "해동된 배아 수",
        "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "미세주입 배아 이식 수",
        "미세주입 후 저장된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "혼합된 난자 수"
    ]
    for c in num_candidates:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # 비율 변수
    if "총 생성 배아 수" in df.columns and "이식된 배아 수" in df.columns:
        df["이식/생성"] = safe_div(df["이식된 배아 수"].fillna(0), df["총 생성 배아 수"].fillna(0))

    if "총 생성 배아 수" in df.columns and "저장된 배아 수" in df.columns:
        df["저장/생성"] = safe_div(df["저장된 배아 수"].fillna(0), df["총 생성 배아 수"].fillna(0))

    if "저장된 배아 수" in df.columns and "해동된 배아 수" in df.columns:
        df["해동/저장"] = safe_div(df["해동된 배아 수"].fillna(0), df["저장된 배아 수"].fillna(0))

    if "미세주입된 난자 수" in df.columns and "미세주입에서 생성된 배아 수" in df.columns:
        df["미세주입_배아/난자"] = safe_div(df["미세주입에서 생성된 배아 수"].fillna(0), df["미세주입된 난자 수"].fillna(0))

    if "이식된 배아 수" in df.columns and "미세주입 배아 이식 수" in df.columns:
        df["미세주입_이식/이식"] = safe_div(df["미세주입 배아 이식 수"].fillna(0), df["이식된 배아 수"].fillna(0))

    # ===== 🆕 추가 Feature 5개 =====
    
    # 1. 배아_품질_점수
    df['배아_품질_점수'] = (
        (df['Day5_이식_여부'] * 3) +
        ((df['이식된 배아 수'].fillna(0).between(1, 2)).astype(int) * 2) +
        ((df['이식/생성'].fillna(0) > 0.5).astype(int))
    )
    
    # 2. 최적_이식_조건
    df['최적_이식_조건'] = (
        (df['Day5_이식_여부'] == 1) & 
        (df['이식된 배아 수'].fillna(0).between(1, 2))
    ).astype(int)
    
    # 3. 고령_여부
    age_map = {
        '만18-34세': 0, '만35-37세': 0, '만38-39세': 0,
        '만40-42세': 1, '만43-44세': 1, '만45-50세': 1,
        '알 수 없음': 0
    }
    df['고령_여부'] = df['시술 당시 나이'].map(age_map).fillna(0).astype(int)
    
    # 4. IVF_과거_성공률
    trials_map = {'0회': 0, '1회': 1, '2회': 2, '3회': 3, '4회': 4, '5회': 5, '6회 이상': 7}
    ivf_trials = df['IVF 시술 횟수'].map(trials_map).fillna(0)
    ivf_success = df['IVF 임신 횟수'].map(trials_map).fillna(0)
    df['IVF_과거_성공률'] = safe_div(ivf_success, ivf_trials)
    
    # 5. 배아_선별률
    df['배아_선별률'] = safe_div(
        (df['총 생성 배아 수'].fillna(0) - df['이식된 배아 수'].fillna(0)),
        df['총 생성 배아 수'].fillna(0)
    )
    
    # ===== 🆕 확정 실패 케이스 (팀원 인사이트) =====
    storage_keywords = ['난자 저장용', '기증용', '배아 저장용', '연구용']
    df['확정_실패_케이스'] = 0
    
    if '배아 생성 주요 이유' in df.columns:
        df['확정_실패_케이스'] = df['배아 생성 주요 이유'].isin(storage_keywords).astype(int)
    
    if '시술 당시 나이' in df.columns:
        df['확정_실패_케이스'] = df['확정_실패_케이스'] | (df['시술 당시 나이'] == '알 수 없음').astype(int)

    return df


train_fe = add_features(train)
test_fe  = add_features(test)


# ============================================================
# 2) Feature/Column split
# ============================================================
drop_cols = [ID_COL, TARGET_COL, "배아_stage_all_missing"]
feature_cols = [c for c in train_fe.columns if c not in drop_cols]

X = train_fe[feature_cols].copy()
y = train_fe[TARGET_COL].copy()
X_test = test_fe[feature_cols].copy()

print(f"\n✅ Total Features: {len(feature_cols)}")

explicit_cat = [
    '시술 시기 코드','시술 당시 나이','시술 유형','특정 시술 유형','배란 자극 여부','배란 유도 유형',
    '배아 생성 주요 이유','총 시술 횟수','클리닉 내 총 시술 횟수','IVF 시술 횟수','DI 시술 횟수',
    '총 임신 횟수','IVF 임신 횟수','DI 임신 횟수','총 출산 횟수','IVF 출산 횟수','DI 출산 횟수',
    '난자 출처','정자 출처','난자 기증자 나이','정자 기증자 나이',
    '시술_대분류','총시술_bin3','나이_3구간',
]
cat_cols = [c for c in explicit_cat if c in X.columns]
cat_cols = sorted(list(set(cat_cols + X.select_dtypes(include=["object"]).columns.tolist())))

num_cols = [c for c in X.columns if c not in cat_cols]

print(f"   범주형: {len(cat_cols)}개")
print(f"   수치형: {len(num_cols)}개")


# ============================================================
# 3) Preprocessing
# ============================================================
def prep_fold(X_tr, X_va, X_te):
    X_tr = X_tr.copy()
    X_va = X_va.copy()
    X_te = X_te.copy()

    for c in cat_cols:
        X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
        X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
        X_te[c] = X_te[c].astype("object").fillna("Unknown").astype(str)

    med = X_tr[num_cols].median(numeric_only=True)
    X_tr[num_cols] = X_tr[num_cols].fillna(med)
    X_va[num_cols] = X_va[num_cols].fillna(med)
    X_te[num_cols] = X_te[num_cols].fillna(med)

    return X_tr, X_va, X_te


# ============================================================
# 4) 3-Model CV: CatBoost + LightGBM + TabNet
# ============================================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_cb = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_tn = np.zeros(len(X))

pred_cb = np.zeros(len(X_test))
pred_lgb = np.zeros(len(X_test))
pred_tn = np.zeros(len(X_test))

print("\n" + "="*80)
print("3-MODEL ENSEMBLE: CatBoost + LightGBM + TabNet")
print("="*80)

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/5")
    print(f"{'='*80}")
    
    X_tr_raw, X_va_raw = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    X_tr, X_va, X_te = prep_fold(X_tr_raw, X_va_raw, X_test)

    pos = (y_tr == 1).sum()
    neg = (y_tr == 0).sum()

    # ---------
    # CatBoost
    # ---------
    print("\n[CatBoost Training...]")
    cb_class_weights = [1.0, (neg / max(pos, 1))]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_cols)
    valid_pool = Pool(X_va, y_va, cat_features=cat_cols)
    test_pool  = Pool(X_te, cat_features=cat_cols)

    cb = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        iterations=8000,
        learning_rate=0.03,
        depth=8,
        l2_leaf_reg=6.0,
        random_strength=1.0,
        subsample=0.8,
        rsm=0.8,
        class_weights=cb_class_weights,
        random_seed=42,
        verbose=0,
        allow_writing_files=False
    )
    cb.fit(train_pool, eval_set=valid_pool, use_best_model=True, early_stopping_rounds=300)

    oof_cb[va_idx] = cb.predict_proba(X_va)[:, 1]
    pred_cb += cb.predict_proba(X_te)[:, 1] / skf.n_splits

    auc_cb = roc_auc_score(y_va, oof_cb[va_idx])

    # ---------
    # LightGBM
    # ---------
    print("[LightGBM Training...]")
    X_tr_lgb = X_tr.copy()
    X_va_lgb = X_va.copy()
    X_te_lgb = X_te.copy()
    for c in cat_cols:
        X_tr_lgb[c] = X_tr_lgb[c].astype("category")
        X_va_lgb[c] = X_va_lgb[c].astype("category")
        X_te_lgb[c] = X_te_lgb[c].astype("category")

    lgb_params = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": 0.03,
        "num_leaves": 128,
        "min_data_in_leaf": 80,
        "feature_fraction": 0.85,
        "bagging_fraction": 0.85,
        "bagging_freq": 1,
        "lambda_l1": 0.0,
        "lambda_l2": 0.0,
        "verbosity": -1,
        "seed": 42,
        "scale_pos_weight": (neg / max(pos, 1)),
    }

    dtr = lgb.Dataset(X_tr_lgb, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
    dva = lgb.Dataset(X_va_lgb, label=y_va, categorical_feature=cat_cols, free_raw_data=False)

    lgbm = lgb.train(
        lgb_params,
        dtr,
        num_boost_round=10000,
        valid_sets=[dva],
        callbacks=[lgb.early_stopping(300, verbose=False)]
    )

    oof_lgb[va_idx] = lgbm.predict(X_va_lgb)
    pred_lgb += lgbm.predict(X_te_lgb) / skf.n_splits

    auc_lgb = roc_auc_score(y_va, oof_lgb[va_idx])

    # ---------
    # TabNet
    # ---------
    if TABNET_AVAILABLE:
        print("[TabNet Training...]")
        
        # 숫자형만 사용 (TabNet은 범주형 직접 처리 어려움)
        X_tr_tn = X_tr[num_cols].values.astype(np.float32)
        X_va_tn = X_va[num_cols].values.astype(np.float32)
        X_te_tn = X_te[num_cols].values.astype(np.float32)
        
        y_tr_arr = y_tr.values
        y_va_arr = y_va.values
        
        tabnet = TabNetClassifier(
            n_d=64,
            n_a=64,
            n_steps=5,
            gamma=1.5,
            n_independent=2,
            n_shared=2,
            lambda_sparse=1e-4,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_params={"step_size": 50, "gamma": 0.9},
            scheduler_fn=torch.optim.lr_scheduler.StepLR,
            mask_type='entmax',
            seed=42,
            verbose=0
        )
        
        tabnet.fit(
            X_tr_tn, y_tr_arr,
            eval_set=[(X_va_tn, y_va_arr)],
            max_epochs=200,
            patience=30,
            batch_size=1024,
            virtual_batch_size=128,
            eval_metric=['auc']
        )
        
        oof_tn[va_idx] = tabnet.predict_proba(X_va_tn)[:, 1]
        pred_tn += tabnet.predict_proba(X_te_tn)[:, 1] / skf.n_splits
        
        auc_tn = roc_auc_score(y_va, oof_tn[va_idx])
    else:
        auc_tn = 0.0

    # ---------
    # Fold summary
    # ---------
    if TABNET_AVAILABLE:
        oof_ens = (oof_cb[va_idx] + oof_lgb[va_idx] + oof_tn[va_idx]) / 3
        auc_ens = roc_auc_score(y_va, oof_ens)
        print(f"\n[Fold {fold}] CB: {auc_cb:.6f} | LGB: {auc_lgb:.6f} | TN: {auc_tn:.6f} | ENS: {auc_ens:.6f}")
    else:
        oof_ens = (oof_cb[va_idx] + oof_lgb[va_idx]) / 2
        auc_ens = roc_auc_score(y_va, oof_ens)
        print(f"\n[Fold {fold}] CB: {auc_cb:.6f} | LGB: {auc_lgb:.6f} | ENS: {auc_ens:.6f}")

# OOF 전체
auc_cb_all  = roc_auc_score(y, oof_cb)
auc_lgb_all = roc_auc_score(y, oof_lgb)

if TABNET_AVAILABLE:
    auc_tn_all = roc_auc_score(y, oof_tn)
    auc_ens_all = roc_auc_score(y, (oof_cb + oof_lgb + oof_tn) / 3)
else:
    auc_tn_all = 0.0
    auc_ens_all = roc_auc_score(y, (oof_cb + oof_lgb) / 2)

print("\n" + "="*80)
print("FINAL OOF SCORES")
print("="*80)
print(f"CatBoost:  {auc_cb_all:.6f}")
print(f"LightGBM:  {auc_lgb_all:.6f}")
if TABNET_AVAILABLE:
    print(f"TabNet:    {auc_tn_all:.6f}")
    print(f"Ensemble:  {auc_ens_all:.6f} (3-model average)")
else:
    print(f"Ensemble:  {auc_ens_all:.6f} (2-model average)")

print(f"\n🎯 Previous Best: 0.740363")
print(f"🚀 Improvement:   +{auc_ens_all - 0.740363:.6f}")


# ============================================================
# 5) Post-processing + Submission
# ============================================================
if TABNET_AVAILABLE:
    pred_test = (pred_cb + pred_lgb + pred_tn) / 3
else:
    pred_test = (pred_cb + pred_lgb) / 2

# 🆕 확정 실패 케이스 강제 0
mask = test_fe['확정_실패_케이스'] == 1
if mask.sum() > 0:
    print(f"\n⚠️  확정 실패 케이스 {mask.sum()}개 발견 → 강제로 0.0 설정")
    pred_test[mask] = 0.0

out = sub.copy()
out[SUB_ID_COL] = test_fe[ID_COL].values
out[SUB_PRED_COL] = pred_test

os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
out.to_csv(OUT_PATH, index=False)

print(f"\n✅ Submission saved: {OUT_PATH}")
print(out.head())
print("\n" + "="*80)
print("ALL-IN-ONE COMPLETE! 🎉")
print("="*80)

✅ 고결측 변수 5개 제거
✅ 고결측 변수 5개 제거

✅ Total Features: 86
   범주형: 24개
   수치형: 62개

3-MODEL ENSEMBLE: CatBoost + LightGBM + TabNet

FOLD 1/5


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:263: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:264: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate


[CatBoost Training...]
[LightGBM Training...]
[TabNet Training...]


/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Early stopping occurred at epoch 59 with best_epoch = 29 and best_val_0_auc = 0.72883


...


[Fold 4] CB: 0.738340 | LGB: 0.737460 | TN: 0.728641 | ENS: 0.738149

FOLD 5/5


/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_tr[c] = X_tr[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:263: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_va[c] = X_va[c].astype("object").fillna("Unknown").astype(str)
/var/folders/zb/7vcy0_ts23q_3pjlgfdd_cl80000gn/T/ipykernel_21294/2635396949.py:264: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecate


[CatBoost Training...]
[LightGBM Training...]
[TabNet Training...]

Early stopping occurred at epoch 86 with best_epoch = 56 and best_val_0_auc = 0.72926


/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



[Fold 5] CB: 0.740367 | LGB: 0.739580 | TN: 0.729257 | ENS: 0.739604

FINAL OOF SCORES
CatBoost:  0.740026
LightGBM:  0.739016
TabNet:    0.728803
Ensemble:  0.739417 (3-model average)

🎯 Previous Best: 0.740363
🚀 Improvement:   +-0.000946

⚠️  확정 실패 케이스 4205개 발견 → 강제로 0.0 설정

✅ Submission saved: ../outputs/06_ML&DL_ensemble_all.csv
           ID  probability
0  TEST_00000     0.003823
1  TEST_00001     0.006125
2  TEST_00002     0.254410
3  TEST_00003     0.183719
4  TEST_00004     0.674107

ALL-IN-ONE COMPLETE! 🎉
